In [29]:
import os
import warnings
import datetime
import numpy as np
import pandas as pd
from dotenv import load_dotenv
from pymongo import MongoClient
from functions import data_loading
from dataprep.eda import create_report

warnings.filterwarnings("ignore")

In [30]:
df = pd.read_pickle('data/temp_fitbit.pkl')
df

,id,date,hour,ecg,heart_rate_alert,type,nightly_temperature,nremhr,spo2,rmssd,...,very_active_minutes,minutes_below_zone_1,minutes_in_zone_1,minutes_in_zone_2,minutes_in_zone_3,water_amount,mood_value,gender,age,bmi
0,621e2ff067b776a2403eb737,2021-12-22,19,NSR,NONE,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FEMALE,<30,>=25
1,621e2ff067b776a2403eb737,2021-11-18,0,NaN,NaN,SKIN,35.025730,NaN,NaN,NaN,...,0.0,1303.0,119.0,0.0,0.0,NaN,NaN,FEMALE,<30,>=25
2,621e2ff067b776a2403eb737,2021-11-18,21,NaN,NaN,SKIN,34.866951,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FEMALE,<30,>=25
3,621e2ff067b776a2403eb737,2021-11-20,0,NaN,NaN,SKIN,35.349583,NaN,NaN,NaN,...,0.0,1400.0,36.0,0.0,0.0,NaN,NaN,FEMALE,<30,>=25
4,621e2ff067b776a2403eb737,2021-11-20,23,NaN,NaN,SKIN,34.495486,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FEMALE,<30,>=25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164490,621e36bb67b776a240b40d64,2022-01-18,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FEMALE,<30,24.0
164491,621e36bb67b776a240b40d64,2022-01-19,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FEMALE,<30,24.0
164492,621e36bb67b776a240b40d64,2022-01-20,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FEMALE,<30,24.0
164493,621e36bb67b776a240b40d64,2022-01-21,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FEMALE,<30,24.0


In [31]:
# stress score preprocessing
stress_df = pd.DataFrame(columns=df.columns)
for user in df['id'].unique():
    user_df = df.loc[df['id']==user]
    for day in user_df['date'].unique():
        day_df = user_df[user_df['date']==day]
        if not day_df['stress_score'].isnull().all():
            value = day_df[~day_df['stress_score'].isna()]['stress_score']
            value = value.iloc[0]
            user_df.loc[user_df['date']==day, 'stress_score'] = value
    stress_df = pd.concat([stress_df, user_df], axis=0, ignore_index=True)
stress_df

,id,date,hour,ecg,heart_rate_alert,type,nightly_temperature,nremhr,spo2,rmssd,...,very_active_minutes,minutes_below_zone_1,minutes_in_zone_1,minutes_in_zone_2,minutes_in_zone_3,water_amount,mood_value,gender,age,bmi
0,621e2ff067b776a2403eb737,2021-12-22,19,NSR,NONE,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FEMALE,<30,>=25
1,621e2ff067b776a2403eb737,2021-11-18,0,NaN,NaN,SKIN,35.025730,NaN,NaN,NaN,...,0.0,1303.0,119.0,0.0,0.0,NaN,NaN,FEMALE,<30,>=25
2,621e2ff067b776a2403eb737,2021-11-18,21,NaN,NaN,SKIN,34.866951,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FEMALE,<30,>=25
3,621e2ff067b776a2403eb737,2021-11-20,0,NaN,NaN,SKIN,35.349583,NaN,NaN,NaN,...,0.0,1400.0,36.0,0.0,0.0,NaN,NaN,FEMALE,<30,>=25
4,621e2ff067b776a2403eb737,2021-11-20,23,NaN,NaN,SKIN,34.495486,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FEMALE,<30,>=25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164490,621e36bb67b776a240b40d64,2022-01-18,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FEMALE,<30,24.0
164491,621e36bb67b776a240b40d64,2022-01-19,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FEMALE,<30,24.0
164492,621e36bb67b776a240b40d64,2022-01-20,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FEMALE,<30,24.0
164493,621e36bb67b776a240b40d64,2022-01-21,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FEMALE,<30,24.0


In [32]:
report = create_report(df, title='Hourly LifeSnaps Overview')
report.show_browser()

  0%|          | 0/9226 [00:00<?, ?it/s]

In [33]:
report = create_report(stress_df, title='Stress preprocessing')
report.show_browser()

  0%|          | 0/9011 [00:00<?, ?it/s]